In [1]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import random

from scipy.fftpack import fft, ifft
from scipy import signal
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

# import features
pd.set_option('precision', 30)
np.set_printoptions(precision = 30)

from keras.models import load_model
from os import listdir
from os.path import isfile, join

Using TensorFlow backend.


In [19]:
model = load_model('model04.hdf5')


In [20]:
train_df = pd.read_csv('diff_format.csv')
float_data=train_df.values[:,:-1]
scaler = preprocessing.StandardScaler()
scaler.fit(float_data)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [21]:
scaler

StandardScaler(copy=True, with_mean=True, with_std=True)

In [22]:
# model.get_weights().shape
# model.layers
def convertdata(float_data, step_size):
    X = float_data[:,0]
#     y = float_data[:,1]
    rows = X.shape[0]//step_size
    X = X[:rows*1000]
    X = X.reshape(rows,1000)
#     y = [y[t] for t in range(999,len(y),1000)]
    return X

def create_FFT_feature(x, fs=4000000):
    fft = abs(np.fft.fft(x))
    timestep = len(x)/fs
    freq = np.fft.fftfreq(len(x), d=timestep)
    i = int(len(x)/2)
    freq = freq[1:i]
    fft = fft[1:i]    
    ind = fft.argmax()
    frequency = freq[ind]
    return frequency, freq, fft

def classic_sta_lta(x, length_sta, length_lta):
    
    sta = np.cumsum(x ** 2)

    # Convert to float
    sta = np.require(sta, dtype=np.float)

    # Copy for LTA
    lta = sta.copy()

    # Compute the STA and the LTA
    sta[length_sta:] = sta[length_sta:] - sta[:-length_sta]
    sta /= length_sta
    lta[length_lta:] = lta[length_lta:] - lta[:-length_lta]
    lta /= length_lta

    # Pad zeros
#     sta[:length_lta - 1] = 0

    # Avoid division by zero by setting zero values to tiny float
    dtiny = np.finfo(0.0).tiny
    idx = lta < dtiny
    lta[idx] = dtiny

    return (sta / lta).mean()

In [24]:
def create_submission(model, test_path):
    segments = [f for f in listdir(test_path) if isfile(join(test_path, f))]
    predictions = pd.DataFrame(columns=['seg_id','time_to_failure'])
#     print(segments)
    for seg_id in segments:
        file = test_path + seg_id
        testdata = pd.read_csv(file).values
        X_matrix = convertdata(testdata, 1000)
#         print(X.shape)
        X_test = np.c_[X_matrix.std(axis=1),
                X_matrix.max(axis=1),
                X_matrix.min(axis=1),
                X_matrix.mean(axis=1),
                np.apply_along_axis(create_FFT_feature, 1, X_matrix, 4000000)[:,0],
                np.apply_along_axis(classic_sta_lta, 1, X_matrix, 10, 200)].astype(np.float)

        X_test = scaler.transform(X_test)    
#         testobj = KerasDataGenerator(testdata)
        
#         test_X = testobj.create_X()
#         X.append(test_X)
        time = model.predict(np.expand_dims(X_test,0))[0][0]
#         print(time)
        df = pd.DataFrame([[seg_id[:-4], time]], columns=['seg_id','time_to_failure'])
        predictions = predictions.append(df, ignore_index=True)
    return predictions

prediction_df = create_submission(model, '../../test/')

In [25]:
prediction_df.to_csv('../../submissions/submission09.csv', index=False)

In [ ]:
model.get_config()